In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 23 10:53:10 2022

@author: Luigi Portinale
"""
from numpy import array
from keras.api.preprocessing.sequence import pad_sequences
from keras.api.models import Sequential
from keras.api.layers import Dense
from keras.api.layers import Flatten
from keras.api.layers import Embedding
from keras.api.layers import Input
#from keras.utils.vis_utils import plot_model

# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels (1: positive; 0: negative)
labels = array([1,1,1,1,1,0,0,0,0,0])

def custom_one_hot(doc, vocab_size):
    words = doc.split()
    encoded = [hash(word) % vocab_size for word in words]
    return encoded

# integer encode the documents
#keras tensorflow one_hot(d,s) is a hash function returning an integer 
#for each word d; it uses a vocabulary size s that should be the number
#of possible words to hash. If this is greater than the actual vocabulary
#size then the probability of collisions is reduced
vocab_size = 50
encoded_docs = [custom_one_hot(d, vocab_size) for d in docs]
print("The encoded documents:")
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# each document is then a vector of 4 integers!
print("The padded documents:")
print(padded_docs)


# define the model
model = Sequential()

#The Embedding has a vocabulary of 50 and an input length of 4. 
#We will choose a small embedding space of 8 dimensions.
#Embedding needs: the vocabulary size, 
#the embedding output dimension (each word will be embedded in a vector
#with this dimension) that is the output size of the layer and 
#finally the length of each document that
#will be provided in input (input size of the layer)
model.add(Input(shape=(max_length,)))
model.add(Embedding(vocab_size, 8))
# each document in input at the above layer will have a vector
#of 8 integers for every word in the document

#now we flatten the output of the embeddings
model.add(Flatten())
#then we use an outout layer for binary classification
model.add(Dense(1, activation='sigmoid'))

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=100, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))